# Experiment 5.0 - Inception Resnet v2 - extract Features

Reproduce Results of [Transfer learning with deep convolutional neural network for liver steatosis assessment in ultrasound images](https://pubmed.ncbi.nlm.nih.gov/30094778/). We used a pre-trained CNN to extract features based on B-mode images. 

The CNNfeatures are extracted using the pretrained Inception-Resnet-v2 implemented in Keras.
See reference: https://jkjung-avt.github.io/keras-inceptionresnetv2/

![Screen Shot 2020-10-12 at 2 57 22 PM](https://user-images.githubusercontent.com/23482039/95781182-47437700-0c9b-11eb-8826-594811ba3322.png)


In [1]:
import sys
import random
sys.path.append('../src')

import warnings
warnings.filterwarnings("ignore") 

import pickle
import pandas as pd

import numpy as np
import mlflow
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tqdm import tqdm

from utils.compute_metrics import get_metrics, get_majority_vote,log_test_metrics
from utils.dataframe_creation import create_dataframe_preproccessing
from sklearn.preprocessing import StandardScaler
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold
from tqdm import tqdm
from pprint import pprint
from itertools import product
from keras.preprocessing.image import ImageDataGenerator


## 1. Feature Extraction

In [9]:
M, N= 434, 636 # ultrasound image dimension
#In the paper, the authors extract the features directly from the pretrained inception resnet_v2
# pooling: Optional pooling mode for feature extraction when include_top is False.
#'max' means that global max pooling will be applied.
net = InceptionResNetV2(include_top=False,
                        weights='imagenet',
                        input_tensor=None,
                        input_shape=(M,N,3),
                        pooling= 'avg')
dataset = create_dataframe_preproccessing()
dataset.head()

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,labels,fat,fname
0,1,0,3,../data/01_raw/raw_images/P1_image1.jpg
1,1,0,3,../data/01_raw/raw_images/P1_image2.jpg
2,1,0,3,../data/01_raw/raw_images/P1_image3.jpg
3,1,0,3,../data/01_raw/raw_images/P1_image4.jpg
4,1,0,3,../data/01_raw/raw_images/P1_image5.jpg


In [10]:
dataset = dataset.astype({"labels": str})
datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_dataframe(dataset, x_col='fname',y_col='labels',target_size=(M,N),shuffle=False, batch_size =25)

Found 550 validated image filenames belonging to 2 classes.


/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
max_pool_dim = 1536
features = np.zeros(shape=(len(dataset), max_pool_dim))
batch_size = 25
assert len(dataset)%batch_size ==0, '550 should be divisable by batch_SIZE'

for i, (inputs_batch, labels_batch) in enumerate(tqdm(generator)):
    features[i*batch_size:(i+1)*batch_size] = net(inputs_batch)
    if (i+1)*batch_size == 550:
        break

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
 95%|█████████▌| 21/22 [08:09<00:23, 23.33s/it]


## 2. Save features

In [16]:
df_features = pd.DataFrame(features)
df_features= pd.concat([dataset['id'], dataset['labels'], df_features], axis=1)
df_features[["labels"]] = df_features[["labels"]].apply(pd.to_numeric)
df_features.head()

,id,labels,0,1,2,3,4,5,6,7,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,1,0,0.134799,0.104123,0.131847,0.034212,0.050750,0.106259,0.184254,0.176089,...,0.005346,0.359540,0.048829,0.134122,0.188153,0.298438,0.014922,0.076330,0.128463,0.296228
1,1,0,0.159228,0.118112,0.171535,0.030628,0.053298,0.121621,0.166092,0.165505,...,0.011214,0.366371,0.056794,0.139955,0.168063,0.249294,0.017226,0.062107,0.132524,0.295283
2,1,0,0.165422,0.175866,0.194926,0.034859,0.043648,0.111044,0.181148,0.240423,...,0.014288,0.416955,0.056556,0.164176,0.212100,0.282104,0.033774,0.072362,0.157818,0.323458
3,1,0,0.176759,0.129491,0.192133,0.034296,0.052507,0.104914,0.152995,0.202268,...,0.010054,0.395699,0.057912,0.149781,0.186323,0.258327,0.036691,0.062423,0.117581,0.330662
4,1,0,0.129541,0.141528,0.184828,0.022645,0.036390,0.122965,0.201552,0.165211,...,0.007299,0.375183,0.044524,0.164961,0.222393,0.247482,0.021502,0.063856,0.130040,0.317397


In [17]:
# Create a dict for the scattering features, and parameters
inception_dict_tensor = {
                'features':df_features,
                'label' : dataset['labels'],
                'pid' : dataset['id'],}

with open('../data/03_features/inception_dict_tensor_avg_pooling.pickle', 'wb') as handle:
    pickle.dump(inception_dict_tensor, handle, protocol=pickle.HIGHEST_PROTOCOL)

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
